# Building a ChatBot

Designing and implementation of an LLM-Powered Chatbot. This chatbot will be able to have a conversation and remember all the previous interractions.

This chatbot will only use the language model to have a conversation

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002127AE76050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002127AE77690>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
chain= model|parser
chain.invoke([HumanMessage(content="Hi, My Name is Mohit. I am a Data scientist")])

"Hi Mohit!\n\nIt's nice to meet you. That's great! Data science is a fascinating field. \n\nWhat kind of data science work do you do?  \n\nAre you working on any interesting projects right now? I'd love to hear more about it.\n"

In [7]:
from langchain_core.messages import AIMessage

model.invoke([HumanMessage(content="Hi, My Name is Mohit. I am a Data Scientist"),
              AIMessage(content="Hi Mohit! It's nice to meet you. That's great! Data science is a fascinating field. What kind of data science work do you do?  \n\nAre you working on any interesting projects right now? I'd love to hear more about it.\n"),
              HumanMessage(content="Hey!, Whats my name and What do I do?")])

AIMessage(content="You told me your name is Mohit and that you are a Data Scientist! 😊  \n\nIs there anything else you'd like to tell me about yourself or your work?  I'm all ears.  👂 \n", response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 105, 'total_tokens': 154, 'completion_time': 0.101318399, 'prompt_time': 0.0074958, 'queue_time': None, 'total_time': 0.108814199}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5003cab7-42ed-49ec-81bf-929c7e5ecfb1-0', usage_metadata={'input_tokens': 105, 'output_tokens': 49, 'total_tokens': 154})

#### Message History

To keep track of inputs and outputs of the model and store them into DB. Future interactions will load these message and pass them into a chain as a part of input

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [16]:
config1={"configurable":{"session_id":"chat1"}}

In [17]:
with_message_history.invoke(
    [HumanMessage(content="Hi, My Name is Mohit. I am a Data Scientist")],
    config=config1
    )

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hello Mohit! It's nice to meet you. \n\nAs a large language model, I'm always interested in learning more about people who work in data science.  What kind of projects are you currently working on?  \n\nAre you working with any interesting datasets or using any new techniques?  \n\nI'm eager to hear more about your work!\n", response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 261, 'total_tokens': 339, 'completion_time': 0.162906195, 'prompt_time': 0.023612229, 'queue_time': None, 'total_time': 0.18651842400000002}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ba200927-f375-461e-a61b-9ede9681c24c-0', usage_metadata={'input_tokens': 261, 'output_tokens': 78, 'total_tokens': 339})

In [18]:
with_message_history.invoke(
    [HumanMessage(content="whats my name")],
    config=config1
    )

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content='Your name is Mohit.  \n\nIs there anything else I can help you with? 😊 \n\n', response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 350, 'total_tokens': 374, 'completion_time': 0.048495836, 'prompt_time': 0.038892171, 'queue_time': None, 'total_time': 0.087388007}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fc9d92cb-2792-44f4-aff0-6bd9567a34db-0', usage_metadata={'input_tokens': 350, 'output_tokens': 24, 'total_tokens': 374})

In [21]:
config2={"configurable":{"session_id":"chat2"}}
result=with_message_history.invoke(
    [HumanMessage(content="whats my name")],
    config=config2
    )
result

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Since I don't have access to any past conversations or personal information, I don't know your name. \n\nWould you like to tell me? 😊  \n", response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 70, 'total_tokens': 107, 'completion_time': 0.075686596, 'prompt_time': 0.004224497, 'queue_time': None, 'total_time': 0.07991109299999999}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6e359de9-6255-47e1-b277-27a7f30b4cab-0', usage_metadata={'input_tokens': 70, 'output_tokens': 37, 'total_tokens': 107})

In [22]:
config2={"configurable":{"session_id":"chat2"}}
result=with_message_history.invoke(
    [HumanMessage(content="my name is Jack")],
    config=config2
    )
result

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content='Nice to meet you, Jack! 👋 \n\nIs there anything I can help you with today? 😊  \n\n', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 119, 'total_tokens': 145, 'completion_time': 0.052935595, 'prompt_time': 0.005101573, 'queue_time': None, 'total_time': 0.058037168}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-72a1b995-61b8-44ab-a88b-01aea2106bfb-0', usage_metadata={'input_tokens': 119, 'output_tokens': 26, 'total_tokens': 145})

In [23]:
config2={"configurable":{"session_id":"chat2"}}
result=with_message_history.invoke(
    [HumanMessage(content="whats my name")],
    config=config2
    )
result

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content='Your name is Jack!  I remember you telling me. 😊  \n\nDo you have another question for me?', response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 155, 'total_tokens': 180, 'completion_time': 0.050536648, 'prompt_time': 0.009155691, 'queue_time': None, 'total_time': 0.059692339000000004}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cb1c66b-4fdc-4153-a3d7-a2be9fbecfc3-0', usage_metadata={'input_tokens': 155, 'output_tokens': 25, 'total_tokens': 180})

In [24]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi, My Name is Mohit. I am a Data Scientist'), AIMessage(content="Hello Mohit! It's nice to meet you.  \n\nAs a large language model, I'm always interested in learning more about people who work in data science. What kind of projects are you currently working on?  \n\nDo you have any specific questions I can help you with today?\n", response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 27, 'total_tokens': 91, 'completion_time': 0.133035318, 'prompt_time': 0.00436504, 'queue_time': None, 'total_time': 0.137400358}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-30f428ef-baf2-4b09-9399-dd115d486df7-0', usage_metadata={'input_tokens': 27, 'output_tokens': 64, 'total_tokens': 91}), HumanMessage(content='Hi, My Name is Mohit. I am a Data Scientist'), AIMessage(content="Hi Mohit, it's great to meet you! \n\nAs a fellow AI, I'm a

## Prompt Template

In [31]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [("system","You are helpful assisstant. Answer all the question to the best of your ability."),
     MessagesPlaceholder(variable_name="messages")])

chain=prompt|model|parser
chain.invoke({"messages":[HumanMessage(content="My Name is Mohit")]})

"Hello Mohit,\n\nIt's nice to meet you!\n\nIs there anything I can help you with today? 😊  \n\n"

In [33]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)
config3={"configurable":{"session_id":"chat3"}}
result=with_message_history.invoke(
    [HumanMessage(content="whats my name")],
    config=config3
    )
result

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'As a helpful assistant, I have no memory of past conversations and do not know your name. Would you like to tell me? 😊  \n\n'

Adding more complexity

In [36]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [("system","You are helpful assisstant. Answer all the question to the best of your ability in {language}"),
     MessagesPlaceholder(variable_name="messages")])

chain=prompt|model|parser
chain.invoke({"messages":[HumanMessage(content="My Name is Mohit")],"language":"French"})

"Enchanté, Mohit ! 👋 \n\nComment puis-je vous aider aujourd'hui ? \n"

In [43]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

config4={"configurable":{"session_id":"chat4"}}
result=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi My name is Mohit")],
    "language":"French"},
    config=config4
    )
result



Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Hola Mohit, \n\nMucho gusto. ¿Cómo puedo ayudarte?  😊 \n'